In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Random effects model

$y_{i,t} = \alpha_i + \beta_i x_{t-1,i} + \gamma_i w_{t-1} + \epsilon_{t,i}$

$\alpha_i = \alpha_0 + \eta_{\alpha_i}$

$\beta_i = \beta_0 + \eta_{\beta_i}$

$\gamma_i = \gamma_0 + \eta_{\gamma_i}$

$\alpha_0 \sim \mathcal{N}(0, \tau_\alpha)$

$\beta_0 \sim \mathcal{N}(0, \tau_\beta)$

$\gamma_0 \sim \mathcal{N}(0, \tau_\gamma)$

$\eta_{\alpha_i} \sim \mathcal{N}(0, \tau_{\eta_\alpha})$

$\eta_{\beta_i} \sim \mathcal{N}(0, \tau_{\eta_\beta})$

$\eta_{\gamma_i} \sim \mathcal{N}(0, \tau_{\eta_\gamma})$

$\tau_\alpha, \tau_\beta, \tau_\gamma, \tau_{\eta_\alpha}, \tau_{\eta_\beta}, \tau_{\eta_\gamma} \sim \Gamma(1,1)$

In [22]:
import pandas as pd
from seminartools.data import read_inflation, read_merged

In [23]:
df = read_merged()
df

,,inflation,gdp_growth,interest_rate,unemployment_rate,commodity_CRUDE_PETRO,commodity_iNATGAS,commodity_iAGRICULTURE,commodity_iMETMIN,commodity_iPRECIOUSMET
country,date,,,,,,,,,
Japan,1977-01-01,0.022951,2.187257,6.333333,2.3,0.00000,0.052425,0.144757,0.041940,0.112740
Canada,1977-01-01,0.023904,1.533830,8.166667,8.8,0.00000,0.052425,0.144757,0.041940,0.112740
United States,1977-01-01,0.017231,1.186233,4.700000,8.2,0.00000,0.052425,0.144757,0.041940,0.112740
Japan,1977-04-01,0.027244,0.703788,5.000000,2.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
Canada,1977-04-01,0.023346,0.510302,7.666667,8.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
...,...,...,...,...,...,...,...,...,...,...
Italy,2023-01-01,0.003938,0.552944,3.000000,8.3,-0.02041,-0.585458,0.002037,0.007418,0.043183
Canada,2023-01-01,0.006080,0.610026,4.500000,5.4,-0.02041,-0.585458,0.002037,0.007418,0.043183
Sweden,2023-01-01,0.014955,0.656897,2.833333,7.8,-0.02041,-0.585458,0.002037,0.007418,0.043183


In [54]:
from seminartools.models.random_effects_model import RandomEffectsModel

model = RandomEffectsModel()
model

In [55]:
df

,,inflation,gdp_growth,interest_rate,unemployment_rate,commodity_CRUDE_PETRO,commodity_iNATGAS,commodity_iAGRICULTURE,commodity_iMETMIN,commodity_iPRECIOUSMET
country,date,,,,,,,,,
Japan,1977-01-01,0.022951,2.187257,6.333333,2.3,0.00000,0.052425,0.144757,0.041940,0.112740
Canada,1977-01-01,0.023904,1.533830,8.166667,8.8,0.00000,0.052425,0.144757,0.041940,0.112740
United States,1977-01-01,0.017231,1.186233,4.700000,8.2,0.00000,0.052425,0.144757,0.041940,0.112740
Japan,1977-04-01,0.027244,0.703788,5.000000,2.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
Canada,1977-04-01,0.023346,0.510302,7.666667,8.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
...,...,...,...,...,...,...,...,...,...,...
Italy,2023-01-01,0.003938,0.552944,3.000000,8.3,-0.02041,-0.585458,0.002037,0.007418,0.043183
Canada,2023-01-01,0.006080,0.610026,4.500000,5.4,-0.02041,-0.585458,0.002037,0.007418,0.043183
Sweden,2023-01-01,0.014955,0.656897,2.833333,7.8,-0.02041,-0.585458,0.002037,0.007418,0.043183


In [56]:
model.fit(df)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [inflation_sigma, Intercept, gdp_growth_lag_1, interest_rate_lag_1, unemployment_rate_lag_1, commodity_CRUDE_PETRO_lag_1, commodity_iNATGAS_lag_1, commodity_iAGRICULTURE_lag_1, commodity_iMETMIN_lag_1, commodity_iPRECIOUSMET_lag_1, inflation_lag_1]


Sampling 4 chains for 300 tune and 500 draw iterations (1_200 + 2_000 draws total) took 79 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


In [57]:
import arviz as az

az.summary(model.results.posterior)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,0.001,0.000,0.001,0.002,0.000,0.0,976.0,1358.0,1.00
gdp_growth_lag_1,0.000,0.000,-0.000,0.000,0.000,0.0,1323.0,1241.0,1.01
interest_rate_lag_1,0.001,0.000,0.001,0.001,0.000,0.0,1869.0,1259.0,1.00
unemployment_rate_lag_1,0.000,0.000,0.000,0.000,0.000,0.0,862.0,739.0,1.01
commodity_CRUDE_PETRO_lag_1,0.008,0.001,0.005,0.010,0.000,0.0,2295.0,1465.0,1.00
commodity_iNATGAS_lag_1,0.003,0.001,0.001,0.004,0.000,0.0,2212.0,1293.0,1.00
commodity_iAGRICULTURE_lag_1,0.016,0.004,0.008,0.023,0.000,0.0,2133.0,1585.0,1.00
commodity_iMETMIN_lag_1,-0.004,0.002,-0.008,0.001,0.000,0.0,1993.0,1725.0,1.00
commodity_iPRECIOUSMET_lag_1,0.008,0.002,0.003,0.012,0.000,0.0,2350.0,1258.0,1.00
inflation_lag_1,0.203,0.021,0.160,0.241,0.001,0.0,923.0,1162.0,1.00


In [59]:
import seaborn as sns
preds = model.predict(df)

preds

inflation
2023-04-01 Australia       0.006286
           Belgium         0.008126
           Canada          0.004396
           Denmark         0.003614
           France          0.003990
           Germany         0.005536
           Italy           0.010781
           Japan           0.002186
           Korea           0.003212
           Netherlands     0.004471
           New Zealand     0.006014
           Norway          0.004066
           Portugal        0.006105
           South Africa    0.009699
           Spain           0.002510
           Sweden          0.008334
           Switzerland     0.000501
           United States   0.003866